# Applied Project in Big Data on Industrial Dataset

## DATA COLLECTION TECHNIQUES
## Part I. Web scraping intro

### 1. Libraries

Let's start from very basic example, we wiil need [urllib](https://docs.python.org/3/library/urllib.html) library for  opening and reading URLs. We will also use [Beautiful Soup](https://beautiful-soup-4.readthedocs.io/en/latest/) Python library to parce HTML data.

In [ ]:
import os
import re
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

### 2. Get text from HTML page

In [ ]:
URL_2_SCRAP = 'https://ai-jobs.net'
print(URL_2_SCRAP)

In [ ]:
request = Request(URL_2_SCRAP)
response = urlopen(request)
html = response.read()

In [ ]:
html

In [ ]:
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
soup

In [ ]:
soup.contents

In [ ]:
soup.contents[0]

In [ ]:
soup.text

In [ ]:
text = soup.text
for ch in ['\n', '\t', '\r']:
    text = text.replace(ch, ' ')

In [ ]:
' '.join(text.split())

### 3. Simple NLP analysis

In [ ]:
# lower text and leave only text without symbols
# with help of regular expressions library

text = re.sub('[^а-яА-Яa-zA-Z]+', ' ', text).strip().lower()
text

In [ ]:
text_as_list = text.split()
text_as_list[:5]

In [ ]:
from collections import Counter

In [ ]:
Counter(text_as_list)

In [ ]:
freqs = dict(Counter(text_as_list))

In [ ]:
freqs

In [ ]:
freqs = dict(
    sorted(
        freqs.items(), 
        key=lambda item: item[1], 
        reverse=True
    )
)
freqs

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
freqs_bar = {k: v for k, v in freqs.items() if v >= 10}

In [ ]:
plt.figure(figsize=(16, 6))
plt.bar(*zip(*freqs_bar.items()))
plt.xticks(rotation='vertical')
plt.show()

### 4. Save HTML to disk as row data

In [ ]:
with open('ai_jobs_page.html', 'w') as file:
    file.write(html.decode())

In [ ]:
with open('ai_jobs_page.html', 'r') as file:
    text = file.read()

In [ ]:
text

### 5. More than text

In [ ]:
soup

In [ ]:
soup.find_all('img')[1:10]

In [ ]:
soup.find_all('img', attrs={'alt': 'Micron Technology logo'})

In [ ]:
soup.find_all('img')[3]['src']

In [ ]:
# or you can use `find` with `attrs`

soup.find('img', attrs={'alt': 'Micron Technology logo'})['src']

In [ ]:
URL_2_SCRAP_IMG = 'https://ai-jobs.net/' + \
    soup.find('img', attrs={'alt': 'Micron Technology logo'})['src']
print(URL_2_SCRAP_IMG)

In [ ]:
request = Request(URL_2_SCRAP_IMG)
response = urlopen(request)
img = response

In [ ]:
img

In [ ]:
from PIL import Image
import numpy as np

In [ ]:
plt.figure(figsize=(12, 8))
img = Image.open(img)
plt.imshow(np.array(img))
plt.show()

In [ ]:
img.save('logo.jpg')

### 6. Get data

In [ ]:
# look carefully at this soup and you will find that 
# job descriptions are insude the `a` tags
# like <a class="col pt-2 pb-3" 
#     href="/job/84649-senior-data-engineer/" 
#     title="View details for this job">

soup

In [ ]:
soup.find('a')

In [ ]:
soup.find_all('a')

In [ ]:
soup.find_all('a', class_='col pt-2 pb-3')

In [ ]:
jobs_list = soup.find_all('a', class_='col pt-2 pb-3')

In [ ]:
# or in `dict` key-value style

jobs_list = soup.find_all('a', {'class': 'col pt-2 pb-3'})

In [ ]:
jobs_list[0]

In [ ]:
jobs_list[1]

In [ ]:
jobs_list[1]['href']

In [ ]:
jobs_list[1]['title']

In [ ]:
jobs_list[1].find('h3')

In [ ]:
jobs_list[1].find('h3').text

In [ ]:
jobs_list[1].find_all('span')

In [ ]:
jobs_list[1].find_all('span')[0]['class']

In [ ]:
# location of the job 

jobs_list[1].find('span', class_='d-block d-md-none text-break')

In [ ]:
# position for the job 

jobs_list[1].find('span', class_='badge rounded-pill text-bg-info my-md-1 d-md-none')

In [ ]:
# requirements

jobs_list[1].find_all('span', class_='badge rounded-pill text-bg-light')

In [ ]:
[x.text 
 for x in jobs_list[1].find_all(
     'span', 
     class_='badge rounded-pill text-bg-light'
 )]

In [ ]:
for job in jobs_list:
    print(job['title'])
    print(URL_2_SCRAP + job['href'])
    print(
        'location ->',
        job.find('span', class_='d-block d-md-none text-break').text
    )
    try:
        print(
            'position ->',
            job.find('span', class_='badge rounded-pill text-bg-info my-md-1 d-md-none').text
        )
    except:
        print('no position defined')
    print(
        'requirements ->',
        [x.text for x in job.find_all('span', class_='badge rounded-pill text-bg-light')]
    )
    print('*' * 50)

Let's try to collect data and put it to dictionary first and then convert to Pandas dataframe.

In [ ]:
from tqdm.notebook import tqdm

# list for all jobs collected
all_jobs = []

for job in tqdm(jobs_list):
    
    # dictionary for a single job
    job_dict = {}
    
    job_dict['description'] = job.find('h3').text
    job_dict['url'] = URL_2_SCRAP + job['href']
    location = job.find(
        'span', 
        class_='d-block d-md-none text-break'
    )
    job_dict['location'] = location.text if location else ''
    time = job.find(
        'span', 
        class_='badge badge-secondary badge-pill my-md-1'
    )
    job_dict['time'] = time.text if time else ''
    position = job.find(
        'span', 
        class_='badge rounded-pill text-bg-secondary my-md-1 ms-1'
    )
    job_dict['position'] = position.text if position else ''
    level = job.find(
        'span', 
        class_='badge rounded-pill text-bg-info my-md-1 d-md-none'
    )
    job_dict['level'] = level.text if level else ''
    salary_range = job.find(
        'span', 
        class_='badge rounded-pill text-bg-info my-md-1 d-none d-md-inline-block'
    )
    job_dict['salary_range'] = salary_range.text if salary_range else ''
    salary = job.find(
        'span', 
        class_='badge rounded-pill text-bg-success d-none d-md-inline-block'
    )
    job_dict['salary'] = salary.text if salary else ''
    job_dict['requirements'] = [
        x.text for x in job.find_all('span', class_='badge rounded-pill text-bg-light')
    ]
    job_dict['offers'] = [
        x.text for x in job.find_all('span', class_='badge rounded-pill text-bg-success')
    ]
    
    all_jobs.append(job_dict)

print('jobs collected:', len(all_jobs))

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(all_jobs)

### 7. Get data by single item

In [ ]:
URL_2_SCRAP_SINGLE = all_jobs[0]['url']
URL_2_SCRAP_SINGLE

In [ ]:
request = Request(URL_2_SCRAP_SINGLE)
response = urlopen(request)
html = response.read()

In [ ]:
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
soup

In [ ]:
soup.find_all('script', {'type': 'application/ld+json'})

In [ ]:
soup.find_all('script', {'type': 'application/ld+json'})[0]

In [ ]:
soup.find_all('script', {'type': 'application/ld+json'})[1]

In [ ]:
import json

In [ ]:
text = soup.find_all('script', {'type': 'application/ld+json'})[1].text
print(text)

In [ ]:
data = json.loads(text)
data

In [ ]:
data['baseSalary']

In [ ]:
data['identifier']['value']

### 8. Saving data

In [ ]:
folder = 'ai_jobs_data'
os.makedirs(folder, exist_ok=True)

In [ ]:
for job in all_jobs[:3]:
    print(job['url'])

In [ ]:
from time import sleep
from random import uniform

In [ ]:
import re

class LazyDecoder(json.JSONDecoder):
    def decode(self, s, **kwargs):
        regex_replacements = [
            (re.compile(r'([^\\])\\([^\\])'), r'\1\\\\\2'),
            (re.compile(r',(\s*])'), r'\1'),
        ]
        for regex, replacement in regex_replacements:
            s = regex.sub(replacement, s)
        return super().decode(s, **kwargs)

In [ ]:
for job in tqdm(all_jobs):
    request = Request(job['url'])
    response = urlopen(request)
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')
    
    text = soup.find_all('script', {'type': 'application/ld+json'})[1].text
    try:
        data = json.loads(text, strict=False)
    except Exception as e:
        data = json.loads(text, cls=LazyDecoder)
        print(e, data)
   
    job_id =  data['identifier']['value']
    file_path = f'{folder}/{job_id}.json'
    with open(file_path, 'w') as file:
        json.dump(data, file)
    
    sleep(uniform(.1, 1.1))

But here is a small problem with collecting more data...